In [1]:
from typing import TypedDict, Annotated
from langgraph.graph import add_messages, StateGraph, END
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage
from dotenv import load_dotenv
# In-memory Checkpointer
from langgraph.checkpoint.memory import MemorySaver

In [2]:
load_dotenv()

True

In [3]:
memory=MemorySaver()

In [4]:
llmLlama=ChatGroq(model="llama-3.1-8b-instant")

In [5]:
class BasicChatState(TypedDict):
    messages: Annotated[list, add_messages]

In [6]:
def chatbot(state: BasicChatState):
    return {
        "messages":[llmLlama.invoke(input=state["messages"])]
    }

In [7]:
graph=StateGraph(state_schema=BasicChatState)

In [8]:
graph.add_node(node="chatbot",action=chatbot)

In [9]:
graph.add_edge(start_key="chatbot",end_key=END)

In [10]:
graph.set_entry_point(key="chatbot")

In [11]:
app=graph.compile(checkpointer=memory)

In [12]:
app.get_graph().print_ascii()

+-----------+  
| __start__ |  
+-----------+  
      *        
      *        
      *        
 +---------+   
 | chatbot |   
 +---------+   
      *        
      *        
      *        
 +---------+   
 | __end__ |   
 +---------+   


In [13]:
config={"configurable":{"thread_id":1}}

In [14]:
response1=app.invoke(input={"messages":HumanMessage(content="Hi I am Harish")},config=config)

In [15]:
response1

{'messages': [HumanMessage(content='Hi I am Harish', additional_kwargs={}, response_metadata={}, id='835bf596-3e52-43ea-9965-8ab6865f8df5'),
  AIMessage(content="Hello Harish, it's nice to meet you. How are you doing today? Is there anything I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 40, 'total_tokens': 73, 'completion_time': 0.047546255, 'prompt_time': 0.003726528, 'queue_time': 0.628732633, 'total_time': 0.051272783}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_c523237e5d', 'finish_reason': 'stop', 'logprobs': None}, id='run--49ecf88e-7c8c-4df0-8328-49d5c767f530-0', usage_metadata={'input_tokens': 40, 'output_tokens': 33, 'total_tokens': 73})]}

In [16]:
response2=app.invoke(input={"messages":HumanMessage(content="What's my name?")},config=config)

In [17]:
response2

{'messages': [HumanMessage(content='Hi I am Harish', additional_kwargs={}, response_metadata={}, id='835bf596-3e52-43ea-9965-8ab6865f8df5'),
  AIMessage(content="Hello Harish, it's nice to meet you. How are you doing today? Is there anything I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 40, 'total_tokens': 73, 'completion_time': 0.047546255, 'prompt_time': 0.003726528, 'queue_time': 0.628732633, 'total_time': 0.051272783}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_c523237e5d', 'finish_reason': 'stop', 'logprobs': None}, id='run--49ecf88e-7c8c-4df0-8328-49d5c767f530-0', usage_metadata={'input_tokens': 40, 'output_tokens': 33, 'total_tokens': 73}),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='68262735-e20c-47d4-a3ea-7e49e9cdd6db'),
  AIMessage(content='Your name is Harish.', additional_kwargs={}, response_metadata={'token

In [18]:
# get snapshot of the latest checkpoint
snapshot=app.get_state(config=config)

In [19]:
snapshot

StateSnapshot(values={'messages': [HumanMessage(content='Hi I am Harish', additional_kwargs={}, response_metadata={}, id='835bf596-3e52-43ea-9965-8ab6865f8df5'), AIMessage(content="Hello Harish, it's nice to meet you. How are you doing today? Is there anything I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 40, 'total_tokens': 73, 'completion_time': 0.047546255, 'prompt_time': 0.003726528, 'queue_time': 0.628732633, 'total_time': 0.051272783}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_c523237e5d', 'finish_reason': 'stop', 'logprobs': None}, id='run--49ecf88e-7c8c-4df0-8328-49d5c767f530-0', usage_metadata={'input_tokens': 40, 'output_tokens': 33, 'total_tokens': 73}), HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='68262735-e20c-47d4-a3ea-7e49e9cdd6db'), AIMessage(content='Your name is Harish.', additional_kwargs={}, response_m

In [22]:
while True:
    user_input=input("User: ")
    if (user_input in ["exit","end"]):
        break
    else:
        result=app.invoke(input={
            "messages":HumanMessage(content=user_input)
        },config=config)
        print(result['messages'][-1].content)

Since my knowledge cutoff is December 2023, I can provide information up to that date. As of my knowledge cutoff, the President of the United States was Joe Biden. However, please note that the presidency may have changed since my knowledge cutoff.
As of my knowledge cutoff in December 2023, Narendra Modi is the Prime Minister of India. He has been serving as the Prime Minister since 2014, and his term has been extended due to the 2019 general elections, where his party, the Bharatiya Janata Party (BJP), won a majority.
I think there might be some confusion earlier. You initially told me your name was Harish, but now you've mentioned it's actually Ritish.
Your name is Ritish.


In [ ]:
while True:
    user_input=input("User: ")
    if (user_input in ["exit","end"]):
        break
    else:
        result=app.invoke(input={
            "messages":HumanMessage(content=user_input)
        },config=config)
        print(result['messages'][-1].content)

User:  What's my name?


Your name is Harish.
